### Libraries used

In [17]:

import numpy as np # for random number distributions
import pandas as pd # for event_log data frame
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
import queue # add FIFO queue data structure
from functools import partial, wraps
import os
import random

import simpy # discrete event simulation environment

### Simulation Tracing

In [8]:
'''        
---------------------------------------------------------
set up event tracing of all simulation program events 
controlled by the simulation environment
that is, all timeout and process events that begin with "env."
documentation at:
  https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing
  https://docs.python.org/3/library/functools.html#functools.partial
'''
def trace(env, callback):
     """Replace the ``step()`` method of *env* with a tracing function
     that calls *callbacks* with an events time, priority, ID and its
     instance just before it is processed.
     note: "event" here refers to simulaiton program events

     """
     def get_wrapper(env_step, callback):
         """Generate the wrapper for env.step()."""
         @wraps(env_step)
         def tracing_step():
             """Call *callback* for the next event if one exist before
             calling ``env.step()``."""
             if len(env._queue):
                 t, prio, eid, event = env._queue[0]
                 callback(t, prio, eid, event)
             return env_step()
         return tracing_step

     env.step = get_wrapper(env.step, callback)

def trace_monitor(data, t, prio, eid, event):
     data.append((t, eid, type(event)))

def test_process(env):
     yield env.timeout(1)

'''
---------------------------------------------------------
set up an event log for recording events
as defined for the discrete event simulation
we use a list of tuples for the event log
documentation at:
  https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing
'''     
def event_log_append(env, caseid, time, activity, event_log):
    event_log.append((caseid, time, activity))
    yield env.timeout(0)

### Simulation Building Blocks

In [61]:
### Arrival and in the queue waiting process

def arrival_waiting(env,sim_params,resources, caseid_queue, event_log):
    # generates new customers... arrivals
    caseid = 0
    while True:  # infinite loop for generating arrivals
        # get the service process started
        # must have waiting customers to begin service
    
        # schedule the time of next customer arrival       
        # by waiting until the next arrival time from now
        # when the process yields an event, the process gets suspended
        inter_arrival_time = round(60*np.random.exponential(scale = sim_params['DINEIN_ARRIVAL_RATE']))
        print("NEXT ARRIVAL TIME: ", env.now + inter_arrival_time)

        yield env.timeout(inter_arrival_time)  # generator function waits
        # env.timeout does not advance the clock. when an event/process calls env.timeout, 
        # that process waits until the clock gets to that time, it does not block 
        # other agents from doing stuff. Other processes can still do stuff 
        # while the first process is waiting for env.timeout to finish.
        caseid += 1
        time = env.now
        activity = 'arrival'
        env.process(event_log_append(env, caseid, time, activity, event_log)) 
        yield env.timeout(0) 
        if caseid_queue.qsize() < sim_params['MAX_QUEUE_LENGTH']:
            caseid_queue.put(caseid)          
            print("Customer joins queue caseid =",caseid,'time = ',env.now,'queue_length =',caseid_queue.qsize())
            time = env.now
            activity = 'join_queue'
            env.process(event_log_append(env, caseid, time, activity, event_log)) 
            env.process(service_process(env, sim_params,resources, caseid_queue, event_log))       
        else:
            print("Customer balks caseid =",caseid,'time = ',env.now,'queue_length =',caseid_queue.qsize()) 
            env.process(event_log_append(env, caseid, env.now, 'balk', event_log))

### Service process

def service_process(env, sim_params,resources, caseid_queue, event_log):
    # is table available and a server available?
    with resources['tables'].request() as table, resources['servers'].request() as server:
            # if so, then the customer can be served
            # the customer is removed from the queue
            # and the customer is seated at a table
            #with caseid_queue.get() as caseid:
                caseid = caseid_queue.get()
                patience = random.uniform(sim_params['MIN_PATIENCE'], sim_params['MAX_PATIENCE'])
                # Wait until a table is free or the customer runs out of patience

                # retrieve the time the customer arrived from the event log
                # to calculate the time the customer waited
                for item in event_log:
                    if item[2] == 'arrival':
                        if item[0] == caseid:
                            arrive = item[1]
                            break

                wait = env.now - arrive

                results = yield table | env.timeout(patience)
                if table in results:
                    results = yield server | env.timeout(patience)
                    if server in results:
                        print("Customer seated and served caseid =",
                                caseid,'time = ',env.now,'wait =',wait)
                        time = env.now
                        activity = 'seated'
                        env.process(event_log_append(env, caseid, time, activity, event_log))
                        time = env.now
                        activity = 'served'
                        env.process(event_log_append(env, caseid, time, activity, event_log))
                        activity = 'eating'
                        yield env.timeout(sim_params['DINEIN_SERVICE_TIME'])
                        time = env.now
                        activity = 'finished_eating'
                        env.process(event_log_append(env, caseid, time, activity, event_log))
                        print("Customer finished eating caseid =",caseid,'time = ',env.now)
                        time = env.now
                        activity = 'leaving'
                        env.process(event_log_append(env, caseid, time, activity, event_log))
                        yield env.timeout(0)
                    else:
                        print("Customer left due to impatience caseid =",caseid,'time = ',env.now)
                        time = env.now
                        activity = 'impatience'
                        env.process(event_log_append(env, caseid, time, activity, event_log))

### Parameters of the simulation

In [67]:
## Restaurant Simulation Constants

# Resources
NUM_SERVERS = 1  # Number of servers
NUM_COOKS = 2  # Number of cooks in the kitchen
NUM_TABLES = 10  # Number of tables in the restaurant
NUM_TAKEOUT_QUEUE = 5  # Number of people that can be queued for takeout before balk
NUM_DINEIN_QUEUE = 5  # Number of people that can be queued for dine-in before balk


# Probabilities parameters
DINE_IN_PROB = 0.5  # Probability that a customer will dine in
TAKEOUT_PROB = 1 - DINE_IN_PROB  # Probability that a customer will order takeout

# Time Parameters
SIMULATION_TIME = 480  # Total simulation time in minutes (e.g., 8 hours)
MIN_PATIENCE = 1  # Minimum time a customer will wait before leaving
MAX_PATIENCE = 2  # Maximum time a customer will wait before leaving
MIN_SEATING_TIME = 1  # Minimum time a customer will spend eating
MAX_SEATING_TIME = 5  # Maximum time a customer will spend eating
MAX_QUEUE_LENGTH = 5  # Maximum number of customers that can be in the queue before balking
DINEIN_SERVICE_TIME = 5  # Average service time for dine-in customers (e.g., 5 minutes)

# rate parameters
DINEIN_ARRIVAL_RATE = 1 / 5  # Average arrival rate for dine-in (e.g., 1 customer every 5 minutes)
TAKEOUT_ARRIVAL_RATE = 1 / 8  # Average arrival rate for takeout (e.g., 1 order every 8 minutes)
BASE_SERVICE_RATE = 1 / 4  # Base service rate for dine-in customers (e.g., service time of 4 minutes)
TAKEOUT_SERVICE_RATE = 1 / 3  # Service rate for takeout orders (e.g., service time of 3 minutes)
KITCHEN_COOK_RATE = 1 / 10  # Average time to cook an order (e.g., 10 minutes per order)

# Random Seed
RANDOM_SEED = 42

OBTAIN_REPRODUCIBLE_RESULTS = True

# dictionary of simulation parameters
sim_params = {
    'NUM_SERVERS': NUM_SERVERS,
    'NUM_COOKS': NUM_COOKS,
    'NUM_TABLES': NUM_TABLES,
    'NUM_TAKEOUT_QUEUE': NUM_TAKEOUT_QUEUE,
    'NUM_DINEIN_QUEUE': NUM_DINEIN_QUEUE,
    'DINE_IN_PROB': DINE_IN_PROB,
    'TAKEOUT_PROB': TAKEOUT_PROB,
    'SIMULATION_TIME': SIMULATION_TIME,
    'MIN_PATIENCE': MIN_PATIENCE,
    'MAX_PATIENCE': MAX_PATIENCE,
    'MIN_SEATING_TIME': MIN_SEATING_TIME,
    'MAX_SEATING_TIME': MAX_SEATING_TIME,
    'MAX_QUEUE_LENGTH': MAX_QUEUE_LENGTH,
    'DINEIN_SERVICE_TIME': DINEIN_SERVICE_TIME,
    'DINEIN_ARRIVAL_RATE': DINEIN_ARRIVAL_RATE,
    'TAKEOUT_ARRIVAL_RATE': TAKEOUT_ARRIVAL_RATE,
    'BASE_SERVICE_RATE': BASE_SERVICE_RATE,
    'TAKEOUT_SERVICE_RATE': TAKEOUT_SERVICE_RATE,
    'KITCHEN_COOK_RATE': KITCHEN_COOK_RATE,
    'RANDOM_SEED': RANDOM_SEED,
    'OBTAIN_REPRODUCIBLE_RESULTS': OBTAIN_REPRODUCIBLE_RESULTS
}

In [37]:
### Simulation Environment

'''
---------------------------------------
set up the SimPy simulation environment
and run the simulation with monitoring
---------------------------------------
'''
if OBTAIN_REPRODUCIBLE_RESULTS: 
    np.random.seed(RANDOM_SEED)


# set up simulation trace monitoring for the simulation
data = []
# bind *data* as first argument to monitor()
this_trace_monitor = partial(trace_monitor, data)

env = simpy.Environment()
trace(env, this_trace_monitor)


p = env.process(test_process(env))

env.run(until=p)

for d in data:
    print(d)



(0, 0, <class 'simpy.events.Initialize'>)
(1, 1, <class 'simpy.events.Timeout'>)
(1, 2, <class 'simpy.events.Process'>)


In [68]:
env = simpy.Environment()

# implement FIFO queue to hold caseid values
caseid_queue = queue.Queue()

# set up limited resource: Tables, Waiters, Cooks

# tables are a limited resource
available_tables = simpy.Resource(env, capacity = NUM_TABLES)

# servers are a limited resource
available_servers = simpy.Resource(env, capacity = NUM_SERVERS)

# cooks are a limited resource
available_cooks = simpy.Resource(env, capacity = NUM_COOKS)

# dictionary of resources
resources = {
    'tables': available_tables,
    'servers': available_servers,
    'cooks': available_cooks
}


caseid = -1  # dummy caseid for beginning of simulation
# dummy caseid values will be omitted from the event_log
# prior to analyzing simulation results

# beginning record in event_log list of tuples of the form
# form of the event_log tuple item: (caseid, time, activity)

event_log = [(caseid,0,'null_start_simulation')]

# event_monitor = partial(event_monitor, event_log)
env.process(event_log_append(env, caseid, env.now, 'start_simulation', event_log))
 
# call customer arrival process/generator to begin the simulation
env.process(arrival_waiting(env,sim_params,resources, caseid_queue, event_log))  

env.run(until = sim_params['SIMULATION_TIME'])  # start simulation with monitoring and fixed end-time

NEXT ARRIVAL TIME:  2
Customer joins queue caseid = 1 time =  2 queue_length = 1
NEXT ARRIVAL TIME:  29
Customer seated and served caseid = 1 time =  2 wait = 0
Customer finished eating caseid = 1 time =  7
Customer joins queue caseid = 2 time =  29 queue_length = 1
NEXT ARRIVAL TIME:  38
Customer seated and served caseid = 2 time =  29 wait = 0
Customer finished eating caseid = 2 time =  34
Customer joins queue caseid = 3 time =  38 queue_length = 1
NEXT ARRIVAL TIME:  58
Customer seated and served caseid = 3 time =  38 wait = 0
Customer finished eating caseid = 3 time =  43
Customer joins queue caseid = 4 time =  58 queue_length = 1
NEXT ARRIVAL TIME:  85
Customer seated and served caseid = 4 time =  58 wait = 0
Customer finished eating caseid = 4 time =  63
Customer joins queue caseid = 5 time =  85 queue_length = 1
NEXT ARRIVAL TIME:  90
Customer seated and served caseid = 5 time =  85 wait = 0
Customer finished eating caseid = 5 time =  90
Customer joins queue caseid = 6 time =  9

In [70]:
# total number of customers served as maximum caseid value
total_customers = max([item[0] for item in event_log])

# total number of customers that left due to impatience
impatience = 0
for item in event_log:
    if item[2] == 'impatience':
        impatience += 1

# total number of customers that left due to balking
balk = 0
for item in event_log:
    if item[2] == 'balk':
        balk += 1

# total number of customers that finished eating
finished_eating = 0
for item in event_log:
    if item[2] == 'finished_eating':
        finished_eating += 1

# print out the results of the simulation
print("Total Customers Served: ", total_customers)
print("Customers Left Due to Impatience: ", impatience)
print("Customers Balked: ", balk)
print("Customers Finished Eating: ", finished_eating)

Total Customers Served:  40
Customers Left Due to Impatience:  9
Customers Balked:  0
Customers Finished Eating:  31
